# Multiple Agents (Group Chat) Demo (Python)

This notebook demonstrates how to interact with **multiple agents** simultaneously using the Python Agent Framework backend API.

## What You'll Learn
- How to send messages to multiple agents at once
- How agents collaborate to answer questions
- Understanding group chat orchestration
- Viewing individual agent contributions
- Working with multi-agent responses

## Prerequisites
Make sure the backend API is running on `http://localhost:8000`

```bash
cd Backend/python
python -m uvicorn main:app --reload
```

## Setup - Install Required Packages

In [1]:
# Install required packages
%pip install requests python-dotenv -q

import requests
import json
from datetime import datetime
from typing import Dict, Any, Optional, List
import time

print("? Packages installed and modules imported successfully!")

Note: you may need to restart the kernel to use updated packages.
? Packages installed and modules imported successfully!



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Configure API Connection

In [2]:
# API Configuration
API_BASE_URL = "http://localhost:8000"
TIMEOUT = 120  # 2 minutes

# Create session for connection pooling
session = requests.Session()

print(f"? Connected to API: {API_BASE_URL}")

? Connected to API: http://localhost:8000


## Step 1: Get Available Agents

In [5]:
# Get all available agents
response = session.get(f"{API_BASE_URL}/agents", timeout=TIMEOUT)
response.raise_for_status()
agents_data = response.json()

print("?? Available Agents for Group Chat:")
print("="*50)

agent_names = [agent.get("name") for agent in agents_data.get("agents", []) if agent.get("name")]
if not agent_names:
    raise ValueError("No agents available from the API")

preferred_order = ["azure_openai_agent", "ms_foundry_people_agent", "bedrock_agent", "openai_agent"]
available_preferred = [a for a in preferred_order if a in agent_names]
if not available_preferred:
    available_preferred = agent_names.copy()

for agent in agents_data.get("agents", []):
    name = agent.get("name", "<unknown>")
    agent_type = agent.get("type", "<unknown>")
    print(f"?? {name} ({agent_type})")

print(f"\n? Total agents available: {len(agent_names)}")

?? Available Agents for Group Chat:
?? azure_openai_agent (azure_openai_agent)
?? ms_foundry_people_agent (ms_people_agent)
?? bedrock_agent (bedrock)
?? openai_agent (openai_agent)

? Total agents available: 4


## Step 2: Send Message to Multiple Agents (User-Friendly Format)

When you specify multiple agents, the framework orchestrates them to work together on your question.

In [10]:
# Prepare chat request for multiple agents using preferred names (align with .NET)
if not agent_names:
    raise ValueError("agent_names is empty; run the agents lookup cell first")

selected = available_preferred[:2] if len(available_preferred) >= 2 else available_preferred
if len(selected) < 2 and len(agent_names) >= 2:
    selected = agent_names[:2]
if len(selected) == 1:
    selected = [selected[0], selected[0]]  # duplicate to keep two slots

multi_agent_request = {
    "message": "What are the latest trends in cloud computing and how do they impact enterprise IT?",
    "agents": selected,
    "max_turns": 3,
    "format": "user_friendly"
}

print("?? Sending message to MULTIPLE agents...")
print(f"Message: {multi_agent_request['message']}")
print(f"Agents: {', '.join(multi_agent_request['agents'])}")
print(f"Max Turns: {multi_agent_request['max_turns']}\n")

# Send the request
response = session.post(
    f"{API_BASE_URL}/chat",
    json=multi_agent_request,
    timeout=TIMEOUT
)
response.raise_for_status()
result = response.json()

# Display synthesized response
print("\n?? Synthesized Response:")
print("="*50)
print(f"Primary Agent: {result.get('agent', '<unknown>')}")
print(f"Session ID: {result.get('session_id', 'N/A')}")
print(f"\nContent:\n{result.get('content', '')}")

# Display metadata
metadata = result.get("metadata") or {}
if metadata:
    print(f"\n?? Collaboration Metadata:")
    print(f"   Agent Count: {metadata.get('agent_count', 'N/A')}")
    print(f"   Total Turns: {metadata.get('total_turns', 'N/A')}")
    print(f"   Is Group Chat: {metadata.get('is_group_chat', 'N/A')}")
    
    if metadata.get("contributing_agents"):
        contributor_list = ", ".join(metadata["contributing_agents"])
        print(f"   Contributing Agents: {contributor_list}")

?? Sending message to MULTIPLE agents...
Message: What are the latest trends in cloud computing and how do they impact enterprise IT?
Agents: azure_openai_agent, ms_foundry_people_agent
Max Turns: 3


?? Synthesized Response:
Primary Agent: azure_openai_agent
Session ID: 8b1969ec-52e6-40c0-9d03-63602dec2f00

Content:
The latest trends in cloud computing are significantly transforming enterprise IT, driving innovation, efficiency, and new capabilities while introducing fresh challenges. As of mid-2024, the most impactful trends include:

**1. Multi-Cloud & Hybrid Cloud Adoption:**  
Enterprises increasingly leverage a mix of public and private clouds, often across multiple providers (e.g., AWS, Azure, Google Cloud). This approach enhances flexibility, resilience, and helps avoid vendor lock-in, but adds complexity in managing interoperability, security, and compliance.

**2. Cloud-Native Technologies:**  
The adoption of containers, Kubernetes, and serverless architectures is accelerati

## Step 3: Send Message with Detailed Format

The detailed format shows each agent's individual contribution to the conversation.

In [11]:
# Request with detailed format to see each agent's contribution
if not agent_names:
    raise ValueError("agent_names is empty; run the agents lookup cell first")

detailed_agents = available_preferred[:2] if len(available_preferred) >= 2 else available_preferred
if len(detailed_agents) < 2 and len(agent_names) >= 2:
    detailed_agents = agent_names[:2]
if len(detailed_agents) == 1:
    detailed_agents = [detailed_agents[0], detailed_agents[0]]

detailed_request = {
    "message": "How can AI and machine learning improve customer service in retail?",
    "agents": detailed_agents,
    "max_turns": 4,
    "format": "detailed"
}

print("?? Sending message with DETAILED format...")
print(f"Message: {detailed_request['message']}")
print(f"Agents: {', '.join(detailed_request['agents'])}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=detailed_request,
    timeout=TIMEOUT
)
response.raise_for_status()
detailed_result = response.json()

# Display conversation details
print("\n?? Detailed Group Chat Response:")
print("="*50)
print(f"Conversation ID: {detailed_result.get('conversation_id', 'N/A')}")
print(f"Total Turns: {detailed_result.get('total_turns', 'N/A')}")

# Show active participants
participants = detailed_result.get("active_participants", [])
print(f"\n?? Active Participants ({len(participants)}):")
for participant in participants:
    print(f"   - {participant}")

# Show individual agent responses
responses = detailed_result.get("responses", [])
print(f"\n?? Individual Agent Contributions ({len(responses)}):")
print("="*80)

for resp in responses:
    agent_name = resp.get("agent", "<unknown>")
    content = resp.get("content", "")
    meta = resp.get("metadata") or {}
    turn = meta.get("turn", "?")
    
    print(f"\n[Turn {turn}] ?? {agent_name}")
    print("-"*80)
    print(content)

# Show summary if available
if detailed_result.get("summary"):
    print(f"\n?? Conversation Summary:")
    print("="*80)
    print(detailed_result["summary"])

# Metadata
metadata = detailed_result.get("metadata") or {}
print(f"\n?? Group Chat Metadata:")
print(f"   Group Chat Type: {metadata.get('group_chat_type')}")
print(f"   Agent Count: {metadata.get('agent_count')}")
print(f"   Response Type: {metadata.get('response_type')}" )

?? Sending message with DETAILED format...
Message: How can AI and machine learning improve customer service in retail?
Agents: azure_openai_agent, ms_foundry_people_agent


?? Detailed Group Chat Response:
Conversation ID: 57ff54da-22ba-45cc-ab6e-5d5b7528476c
Total Turns: 3

?? Active Participants (0):

?? Individual Agent Contributions (3):

[Turn 1] ?? azure_openai_agent
--------------------------------------------------------------------------------
AI and machine learning can significantly enhance customer service in retail by streamlining processes, personalizing experiences, and increasing efficiency. Here are some key ways these technologies can improve customer service:

### 1. Personalized Recommendations
- **Product Suggestions**: AI analyzes customer behavior, purchase history, and preferences to offer tailored product recommendations, increasing satisfaction and sales.
- **Dynamic Promotions**: Machine learning can identify which offers or discounts are most likely to appe

## Step 4: Complex Query with Multiple Agents

Try a more complex question that requires different types of expertise.

In [12]:
# Complex query requiring multiple perspectives
if not agent_names:
    raise ValueError("agent_names is empty; run the agents lookup cell first")

complex_agents = available_preferred[:2] if len(available_preferred) >= 2 else available_preferred
if len(complex_agents) < 2 and len(agent_names) >= 2:
    complex_agents = agent_names[:2]
if len(complex_agents) == 1:
    complex_agents = [complex_agents[0], complex_agents[0]]

complex_request = {
    "message": """I need help understanding: 
        1. What is Azure OpenAI Service?
        2. How can it be integrated with enterprise applications?
        3. What are the security considerations?""",
    "agents": complex_agents,
    "max_turns": 5,
    "format": "user_friendly"
}

print("?? Sending complex multi-part question...")
print(f"Agents: {', '.join(complex_request['agents'])}\n")

response = session.post(
    f"{API_BASE_URL}/chat",
    json=complex_request,
    timeout=TIMEOUT
)
response.raise_for_status()
complex_result = response.json()

print("\n?? Comprehensive Response:")
print("="*50)
print(complex_result.get("content", ""))

# Show collaboration stats
metadata = complex_result.get("metadata") or {}
print(f"\n?? Collaboration Statistics:")
print(f"   Agents Participated: {metadata.get('agent_count')}")
print(f"   Total Conversation Turns: {metadata.get('total_turns')}")

if metadata.get("contributing_agents"):
    print(f"   Contributors:")
    for contrib in metadata["contributing_agents"]:
        print(f"      - {contrib}")

?? Sending complex multi-part question...
Agents: azure_openai_agent, ms_foundry_people_agent


?? Comprehensive Response:
Absolutely, here’s a concise, unified overview addressing your three questions about Azure OpenAI Service:

---

**1. What is Azure OpenAI Service?**

Azure OpenAI Service is a Microsoft Azure cloud offering that provides secure, scalable access to OpenAI’s advanced language models (such as GPT-4, DALL-E, and Codex) via REST APIs and SDKs. It allows enterprises to build AI-powered solutions for tasks like natural language processing, text and image generation, and code completion, all managed within Azure’s enterprise-grade environment.

---

**2. How can it be integrated with enterprise applications?**

Integration is flexible and can be achieved through:
- **REST APIs & SDKs:** Connect from web, mobile, desktop, or backend applications using secure HTTP calls or supported programming languages (Python, C#, JavaScript, etc.).
- **Azure Ecosystem:** Use services li

## Step 5: Compare Single vs Multiple Agent Responses

Let's compare how a single agent vs multiple agents handle the same question.

In [13]:
test_question = "What are the benefits and challenges of implementing microservices architecture?"

if not agent_names:
    raise ValueError("agent_names is empty; run the agents lookup cell first")

selected = available_preferred[:2] if len(available_preferred) >= 2 else available_preferred
if len(selected) < 2 and len(agent_names) >= 2:
    selected = agent_names[:2]
if len(selected) == 1:
    selected = [selected[0], selected[0]]
primary_agent, secondary_agent = selected[0], selected[1]

print(f"Test Question: {test_question}")
print("\n" + "="*80)

# Test 1: Single Agent
print("\n?? Test 1: SINGLE AGENT Response")
print("-"*80)

single_agent_req = {
    "message": test_question,
    "agents": [primary_agent],
    "format": "user_friendly"
}

single_resp = session.post(
    f"{API_BASE_URL}/chat",
    json=single_agent_req,
    timeout=TIMEOUT
)
single_resp.raise_for_status()
single_result = single_resp.json()

single_agent_name = single_result.get("agent", primary_agent)
print(f"Agent: {single_agent_name}")
single_content = single_result.get("content", "")
print(f"Response Length: {len(single_content)} characters")
print(f"\n{single_content[:300]}...")

# Test 2: Multiple Agents
print("\n\n?? Test 2: MULTIPLE AGENTS Response")
print("-"*80)

multi_agent_req = {
    "message": test_question,
    "agents": [primary_agent, secondary_agent] if len(agent_names) > 1 else [primary_agent],
    "max_turns": 3,
    "format": "user_friendly"
}

multi_resp = session.post(
    f"{API_BASE_URL}/chat",
    json=multi_agent_req,
    timeout=TIMEOUT
)
multi_resp.raise_for_status()
multi_result = multi_resp.json()

multi_meta = multi_result.get("metadata") or {}
print(f"Agents Involved: {multi_meta.get('agent_count')}")
print(f"Total Turns: {multi_meta.get('total_turns')}")

multi_content = multi_result.get("content", "")
print(f"Response Length: {len(multi_content)} characters")
print(f"\n{multi_content[:300]}...")

print("\n\n?? Comparison Summary:")
print("="*80)
print(f"Single Agent: {len(single_content)} chars, 1 agent ({single_agent_name})")
print(f"Multiple Agents: {len(multi_content)} chars, {multi_meta.get('agent_count')} agents, {multi_meta.get('total_turns')} turns")
print(f"\n?? Multiple agents can provide more comprehensive responses through collaboration!")

Test Question: What are the benefits and challenges of implementing microservices architecture?


?? Test 1: SINGLE AGENT Response
--------------------------------------------------------------------------------
Agent: azure_openai_agent
Response Length: 3173 characters

**Microservices architecture** is an approach to software development in which applications are built as a collection of small, independent services that communicate over APIs. This model offers several benefits but also presents notable challenges.

---

## **Benefits of Microservices Architecture*...


?? Test 2: MULTIPLE AGENTS Response
--------------------------------------------------------------------------------
Agents Involved: 2
Total Turns: 3
Response Length: 2600 characters

**Microservices architecture** structures an application as a suite of small, independently deployable services, each focused on a specific business capability and communicating via APIs. This approach offers several advantages, but also

## Step 6: Get Available Group Chat Templates

The API provides pre-configured templates for common multi-agent scenarios.

In [14]:
# Get available templates
try:
    response = session.get(f"{API_BASE_URL}/chat/templates", timeout=TIMEOUT)
    response.raise_for_status()
    templates_data = response.json()
    
    print("?? Available Group Chat Templates:")
    print("="*50)
    
    templates = templates_data.get("templates", [])
    
    for template in templates:
        name = template.get("name", "<unknown>")
        description = template.get("description", "")
        agent_count = template.get("agent_count", "?")
        
        print(f"\n?? Template: {name}")
        print(f"   Description: {description}")
        print(f"   Agents: {agent_count}")
        
        if template.get("agents"):
            agent_list = ", ".join(template["agents"])
            print(f"   Agent Names: {agent_list}")
except Exception as ex:
    print(f"??  Templates endpoint not available: {ex}")

??  Templates endpoint not available: 404 Client Error: Not Found for url: http://localhost:8000/chat/templates


## Summary

In this notebook, you learned:
- ? How to send messages to multiple agents simultaneously
- ? Understanding group chat orchestration and collaboration
- ? Viewing individual agent contributions vs synthesized responses
- ? Comparing single-agent vs multi-agent responses
- ? Using detailed format to see the conversation flow
- ? Exploring pre-configured group chat templates

## Key Takeaways
- Multiple agents can provide more comprehensive and nuanced responses
- The framework automatically orchestrates agent collaboration
- You can choose between synthesized (user-friendly) or detailed responses
- Templates provide pre-configured multi-agent scenarios

## Memory Feature (NEW)
You can now enable long-running memory across all agent types:
- Add `enable_memory: True` to any chat request (single or multi-agent)
- Agents will extract and remember your name and persona
- See **04-LongRunningMemory-Demo.ipynb** for complete examples

**Example with Memory:**
```python
{
  "message": "My name is Alice and I'm a data scientist",
  "agents": ["generic_agent", "knowledge_finder"],
  "enable_memory": True,
  "max_turns": 3
}
```

## Next Steps
- Try the **03-ContentSafety-Demo** to see how content is moderated
- Check out the **04-LongRunningMemory-Demo** for memory and session management
- Explore the **single agent demo** for basic agent interaction